In [ ]:
# 開啟帶有密碼的excel使用方式
df = io.BytesIO()

with open(path, 'rb') as f:
    encrypted_file = msoffcrypto.OfficeFile(f)
    password = input('Please enter the password.\n')
    encrypted_file.load_key(password=password)
    encrypted_file.decrypt(df)
    file_name = input('Please enter the file name:\n')
    vars()[file_name] = pd.read_excel(df)

In [ ]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import io
import msoffcrypto
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import plotly.express as px
import calendar
def my_formatter(x):
    if x < 0:
        return '{:,.0f}'.format(-x)
    return '{:,.0f}'.format(x)
pd.options.display.float_format = my_formatter

In [ ]:
category = [calendar.month_name[i] for i in range(1, 13)]

In [ ]:
class DataExtrct:
    files = {}
    
    def __init__(self):
        pass
    
    def get_path(self):
        root = tk.Tk()
        root.withdraw()
        path = filedialog.askopenfilename()
        self.path = path
        return path
    
    def open_file(self):
        df = io.BytesIO()
        with open(self.path, 'rb') as f:
            encrypted_file = msoffcrypto.OfficeFile(f)
            password = input('Please enter the password:\n')
            encrypted_file.load_key(password=password)
            encrypted_file.decrypt(df)
            file_name = input('Please enter the file name:\n')
            self.files.setdefault(file_name, pd.read_excel(df))
    
    def file_load(self):
        self.get_path()
        self.open_file()

In [ ]:
class ProductionSalesReport:
    
    def __init__(self):
        self.path = self.__getFilePath()
        self.df = self.__open_file(self.path)
        
    def __repr__(self):
        return repr(self.df)
    
    def __getFilePath(self):
        '''
        取得開啟檔案路徑
        '''
        root = tk.Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename()
        return file_path

    def __open_file(self, path):
        '''
        path: str
        開啟檔案，如果工作表超過一個需要選取
        '''
        df = pd.read_excel(path, None)
        return df

In [ ]:
budget = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx", sheet_name='2021預算總表 (revised)')

In [ ]:
budget = ProductionSalesReport()

In [ ]:
budget = budget.df

In [ ]:
sales = ProductionSalesReport()

In [ ]:
sales = sales.df

In [ ]:
sales = sales['CS OPO']

In [ ]:
budget = budget[budget['類型'] == '天線']

In [ ]:
budget['類型'] = '預算'

In [ ]:
budget['BU'] = budget.apply(lambda x: 'RF2' if x['品名'].startswith('RFDP') else 'RF1', axis=1)

In [ ]:
budget = budget.groupby(['類型', '負責業務', 'BU', '預交年份', '預交月份'])[['數量', '本國幣別NTD']].sum().reset_index()

In [ ]:
sales['更新日期'] = sales['更新日期'].dt.strftime('%Y-%m-%d')

In [ ]:
sales = sales[sales['更新日期'] == sales['更新日期'].max()]

In [ ]:
sales['預交月份'] = sales['預交日期'].dt.month_name()

In [ ]:
sales['預交年份'] = sales['預交日期'].dt.year

In [ ]:
sales['類型'] = '預估實績'

In [ ]:
sales['負責業務'] = sales['負責業務'].map(lambda x: '許凱智' if x in ['楊婉芬', '周彥宏'] else x)

In [ ]:
sales = sales[(sales['狀態'].str.contains('出')) & (sales['預交日期'].between('2021-01-01', '2021-12-31')) & (sales['BU'] == 'RF1') & (sales['負責業務'].isin(['許凱智', '墨漢雷迪', '鄭里緗']))]

In [ ]:
sales['數量'] = sales.apply(lambda x: x['數量'] * 1000 if x['銷售單位'] == 'KPCS' else x['數量'], axis=1)

In [ ]:
sales = sales.groupby(['類型', '負責業務', 'BU', '預交年份', '預交月份'])[['數量', '集團匯率*金額']].sum().reset_index()

In [ ]:
sales.columns = ['類型', '負責業務', 'BU', '預交年份', '預交月份', '數量', '本國幣別NTD']

In [ ]:
result = pd.concat([budget, sales], axis=0)

In [ ]:
result['預交月份'] = pd.Categorical(result['預交月份'], categories=category, ordered=True)

In [ ]:
result

In [ ]:
px.histogram(result, x='預交月份', y='本國幣別NTD', color='類型', 
             facet_col='負責業務', barmode='group', category_orders={'預交月份': category}, 
             log_y=True, labels=dict(預交月份='月份', 本國幣別NTD='金額'), title='2021年各月金額')

In [ ]:
result.to_excel(r"D:\OneDrive\INPAQ\報告簡報類\產銷會議\預算實績更新總表.xlsx", index=False)

### 未結訂單庫存表

In [ ]:
stock = ProductionSalesReport()

In [ ]:
stock = stock.df

In [ ]:
stock['更新日期'] = stock['更新日期'].dt.strftime('%Y-%m-%d')

In [ ]:
x = stock['更新日期'].unique()[-2:].tolist()

In [ ]:
last_two_week = stock[stock['更新日期'].isin(x)]

In [ ]:
pd.pivot_table(last_two_week, index=['更新日期', '業務人員名稱(NEW)'], columns=['庫存區間'], values=['批次庫存數', '銷售金額(NTD)\n'], aggfunc='sum', fill_value=0)

In [ ]:
df = pd.read_excel(path)

In [ ]:
df['未交量'] = df.apply(lambda x: x['未交量'] * 1000 if x['單位'] == 'KPCS' else x['未交量'], axis=1)

In [ ]:
df['批次庫存數'] = df.apply(lambda x: x['批次庫存數'] * 1000 if x['單位'] == 'KPCS' else x['批次庫存數'], axis=1)

In [ ]:
df['單位'] = df['單位'].map(lambda x: 'PCS' if x == 'KPCS' else x)

In [ ]:
import plotly.express as px

In [ ]:
df.columns

In [ ]:
result = df.groupby(['庫存區間', '業務人員名稱(NEW)', '品名', '送貨客戶名稱'])[['批次庫存數', '未交量', '批次庫存金額NTD', '銷售金額(NTD)\n']].sum().reset_index()

In [ ]:
df.to_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\報告簡報類\產銷會議\20210712\整理過的未結訂單.xlsx", index=False)

## Susan OPO

In [ ]:
path = getFilePath()

In [ ]:
df = pd.read_excel(path, sheet_name='Susan提供的OPO')

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df.columns

## CS OPO

In [ ]:
path = getFilePath()
df = pd.read_excel(path, sheet_name='CS OPO')

In [ ]:
df.columns

In [ ]:
df = df.drop_duplicates(subset=['銷售單號', '銷售項次', '客戶訂單', '客戶訂單項次'], keep='last')

In [ ]:
df.to_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\報告簡報類\產銷會議\20210712\整理過的CS OPO.xlsx", index=False)

## 累積實績（使用weekly report的數據）

In [ ]:
df = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx", sheet_name='出貨明細')

In [ ]:
df.columns

In [ ]:
answer = df[(df['BU'] == 'RF1') & (df['狀態'].str.contains('出')) & (df['預交日期'].between('2021-01-01', '2021-09-30')) & (~df['出通單號'].isna())]

In [ ]:
answer.groupby(['負責業務'])[['本國幣別NTD']].sum()